<a href="https://colab.research.google.com/github/VihaanMotwani/Automated-Diabetic-Foot-Screening-System/blob/main/Color_Based_Segmentation_in_HSV.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [12]:
import cv2
import glob
import os
import numpy as np

In [ ]:
# Adjust these to your own folder paths
input_folder = "images"
output_folder = "output"

# Create the output folder if it doesn't exist
os.makedirs(output_folder, exist_ok=True)

# Define HSV range for the foot (example range—adjust as needed)
lower_color = np.array([0, 40, 40], dtype=np.uint8)
upper_color = np.array([50, 255, 255], dtype=np.uint8)

# Morphological kernel
kernel = np.ones((5, 5), np.uint8)

In [31]:
# Loop over all image files in the input folder
for img_path in glob.glob(os.path.join(input_folder, "*.jpg")):
    # Read the image
    img = cv2.imread(img_path)
    if img is None:
        print(f"Could not read image: {img_path}")
        continue

    # (Optional) Crop out a known region like the scale bar, if needed
    # Example: Crop 50 px from the right
    # img = img[:, :-50]

    # Convert to HSV
    hsv = cv2.cvtColor(img, cv2.COLOR_BGR2HSV)

    # Create a binary mask
    mask = cv2.inRange(hsv, lower_color, upper_color)

    # Morphological closing to fill small holes
    closed_mask = cv2.morphologyEx(mask, cv2.MORPH_CLOSE, kernel)

    # Keep only the largest connected component (the foot)
    num_labels, labels, stats, centroids = cv2.connectedComponentsWithStats(closed_mask, connectivity=8)
    # If there's at least one foreground component
    if num_labels > 1:
        # Find the label of the largest component (excluding background at index 0)
        largest_label = 1 + np.argmax(stats[1:, cv2.CC_STAT_AREA])
        largest_mask = np.uint8(labels == largest_label) * 255
    else:
        # If no foreground found, just keep the closed_mask
        largest_mask = closed_mask

    # Extract the foot region
    foot_segmented = cv2.bitwise_and(img, img, mask=largest_mask)

    # Save the segmented foot image
    filename = os.path.basename(img_path)
    out_path = os.path.join(output_folder, f"seg_{filename}")
    cv2.imwrite(out_path, foot_segmented)
    print(f"Processed and saved: {out_path}")


Processed and saved: output/seg_1017085011-IR.jpg
Processed and saved: output/seg_1017084959-IR.jpg
Processed and saved: output/seg_1017084940-IR.jpg
Processed and saved: output/seg_1017084927-IR.jpg
Processed and saved: output/seg_1017084836-IR.jpg
Processed and saved: output/seg_1017084917-IR.jpg
Processed and saved: output/seg_1017084950-IR.jpg
Processed and saved: output/seg_1017084827-IR.jpg
